# EDS Block Experimentation - Context Aware

This notebook works in both JSLab (Node.js) and browser environments.

## Environment Detection

- **JSLab/Node.js**: Sets up jsdom virtual DOM and helper functions
- **Browser (ipynb-viewer)**: Uses native browser APIs directly

## Important Notes

1. **In JSLab**: Run Cell 1 first to initialize the environment
2. **In Browser**: Cell 1 auto-detects and skips Node.js setup
3. **Pure EDS blocks** - Works best with vanilla JavaScript blocks
4. **Context-aware execution** - Automatically detects Node.js vs browser
5. **Visual output** - Generates live preview HTML with iframe controls
6. **Iframe previews** - Open `-live-preview.html` files for interactive testing

In [ ]:
// ============================================================================
// SETUP: Context-aware initialization
// Run this cell FIRST - works in both JSLab and Browser!
// ============================================================================

// Detect execution environment
const isNode = typeof process !== 'undefined' && process.versions && process.versions.node;
const isBrowser = typeof window !== 'undefined' && typeof document !== 'undefined';

console.log('Environment:', isNode ? 'Node.js (JSLab)' : 'Browser');

// ============================================================================
// NODE.JS SETUP (JSLab/Jupyter)
// ============================================================================
if (isNode) {
  // Load jsdom and set up virtual DOM
  const { JSDOM } = require('jsdom');
  
  // Create virtual DOM
  const dom = new JSDOM('<!DOCTYPE html><html><head></head><body></body></html>', {
    url: 'http://localhost',
    pretendToBeVisual: true
  });
  
  // Make DOM globals available
  global.document = dom.window.document;
  global.window = dom.window;
  global.HTMLElement = dom.window.HTMLElement;
  global.Element = dom.window.Element;
  global.Node = dom.window.Node;
  global.customElements = dom.window.customElements;
  global.CustomEvent = dom.window.CustomEvent;
  global.Event = dom.window.Event;
  
  console.log('✓ Virtual DOM environment initialized');
  
  // Ensure output directory exists
  const fs = require('fs');
  const outputDir = './ipynb-tests';
  if (!fs.existsSync(outputDir)) {
    fs.mkdirSync(outputDir, { recursive: true });
    console.log(`✓ Created output directory: ${outputDir}`);
  } else {
    console.log(`✓ Output directory ready: ${outputDir}`);
  }
  
  // Load helper functions from external module
  const path = require('path');
  const helpersPath = path.resolve('./scripts/ipynb-helpers.js');
  
  // Dynamic import of ES modules
  const helpers = await import(helpersPath);
  global.loadBlockStyles = helpers.loadBlockStyles;
  global.testBlock = helpers.testBlock;
  global.saveBlockHTML = helpers.saveBlockHTML;
  global.createIframePreview = helpers.createIframePreview;
  
  console.log('✓ Loaded helper functions from scripts/ipynb-helpers.js');
  console.log('✓ Available functions:');
  console.log('  - global.testBlock(blockName, innerHTML)');
  console.log('  - global.saveBlockHTML(blockName, innerHTML, filename, options)');
  console.log('  - global.createIframePreview(blockName, blockHTML)');
  console.log('  - global.loadBlockStyles(blockName)');
  
  console.log('✓ Node.js environment ready');
}

// ============================================================================
// BROWSER SETUP
// ============================================================================
else if (isBrowser) {
  console.log('✓ Browser environment detected');
  console.log('✓ Using native browser APIs');
  
  // Browser helpers use native APIs
  window.testBlock = async function(blockName, innerHTML = '') {
    console.log(`Testing: ${blockName}`);
    
    const block = document.createElement('div');
    block.className = blockName;
    
    if (innerHTML) {
      block.innerHTML = innerHTML;
    }
    
    // In browser, we could try to load the block's decorate function
    try {
      const module = await import(`/blocks/${blockName}/${blockName}.js`);
      if (module.default) {
        await module.default(block);
        console.log('✓ Block decorated');
      }
    } catch (e) {
      console.log('ℹ Block decoration skipped:', e.message);
    }
    
    return block;
  };
  
  // Visual helper for browser - creates styled container
  window.displayBlock = function(block) {
    const container = document.createElement('div');
    container.style.cssText = 'margin: 20px 0; padding: 20px; background: white; border-radius: 8px; box-shadow: 0 2px 8px rgba(0,0,0,0.1);';
    container.appendChild(block);
    return container;
  };
  
  // Create iframe preview HTML - browser version
  window.createIframePreview = function(blockName, blockHTML) {
    const previewHTML = `<!DOCTYPE html>
<html>
<head>
  <meta charset="UTF-8">
  <title>Live Preview - ${blockName}</title>
  <style>
    * { margin: 0; padding: 0; box-sizing: border-box; }
    body { font-family: system-ui, -apple-system, sans-serif; background: #1e1e1e; color: #fff; overflow: hidden; }
    .preview-wrapper { position: fixed; top: 0; left: 0; width: 100%; height: 100%; display: flex; flex-direction: column; }
    .preview-header { background: #2d2d2d; padding: 12px 20px; display: flex; justify-content: space-between; align-items: center; border-bottom: 1px solid #3e3e3e; box-shadow: 0 2px 8px rgba(0,0,0,0.3); }
    .preview-title { font-size: 14px; font-weight: 500; color: #cccccc; }
    .preview-title strong { color: #4fc3f7; }
    .preview-controls { display: flex; gap: 8px; align-items: center; }
    .btn { padding: 6px 12px; border: none; border-radius: 4px; font-size: 12px; cursor: pointer; transition: all 0.2s; font-weight: 500; }
    .btn-refresh { background: #4fc3f7; color: #1e1e1e; }
    .btn-refresh:hover { background: #29b6f6; }
    .btn-close { background: #f44336; color: white; }
    .btn-close:hover { background: #d32f2f; }
    .preview-content { flex: 1; overflow: auto; background: #f5f5f5; padding: 20px; }
    .preview-container { max-width: 1200px; margin: 0 auto; background: white; padding: 40px; border-radius: 8px; box-shadow: 0 2px 8px rgba(0,0,0,0.1); }
    .status { font-size: 11px; color: #888; padding: 0 8px; }
  </style>
</head>
<body>
  <div class="preview-wrapper">
    <div class="preview-header">
      <div class="preview-title">🔴 LIVE PREVIEW: <strong>${blockName}</strong> Block</div>
      <div class="preview-controls">
        <span class="status">Interactive Preview</span>
        <button class="btn btn-refresh" onclick="location.reload()">↻ Refresh</button>
        <button class="btn btn-close" onclick="closePreview()">✕ Close</button>
      </div>
    </div>
    <div class="preview-content">
      <div class="preview-container">
        <h2>${blockName} Block Preview</h2>
        ${blockHTML}
      </div>
    </div>
  </div>
  <script>
    function closePreview() {
      if (window.opener) {
        window.close();
      } else {
        history.back();
      }
    }
    document.addEventListener('keydown', (e) => {
      if (e.key === 'Escape') closePreview();
    });
  </script>
</body>
</html>`;
    return previewHTML;
  };
  
  // Open iframe preview in new window
  window.openIframePreview = function(blockName, blockHTML) {
    const previewHTML = window.createIframePreview(blockName, blockHTML);
    const blob = new Blob([previewHTML], { type: 'text/html' });
    const url = URL.createObjectURL(blob);
    const win = window.open(url, '_blank', 'width=1200,height=800');
    console.log('✓ Opened iframe preview in new window');
    return win;
  };
  
  console.log('✓ Browser helpers ready');
  console.log('✓ Available: window.testBlock(), window.displayBlock()');
  console.log('✓ Available: window.createIframePreview(), window.openIframePreview()');
}

console.log('\n========================================');
console.log('Setup complete! Ready to test EDS blocks');
console.log('========================================\n');

'Setup complete!'

## Part 1: Simple Tests

Test basic DOM functionality that works in both environments.

In [ ]:
// Context-aware DOM test
const isNode = typeof process !== 'undefined' && process.versions?.node;
const doc = isNode ? global.document : document;

const testDiv = doc.createElement('div');
testDiv.textContent = 'Hello from ' + (isNode ? 'Node.js (JSLab)' : 'Browser (ipynb-viewer)') + '!';
testDiv.className = 'test';
testDiv.style.cssText = 'padding: 20px; background: #e3f2fd; border-radius: 8px; color: #1976d2; font-weight: bold;';

console.log('✓ Created element:', testDiv.outerHTML);

// Return for display
testDiv.outerHTML

## Part 2: Testing Blocks with Content

Test blocks that require specific content structures.

In [ ]:
// Context-aware block testing
const isNode = typeof process !== 'undefined' && process.versions?.node;
const testBlockFn = isNode ? global.testBlock : window.testBlock;

// Example 1: Test HelloWorld block (no content needed)
const helloBlock = await testBlockFn('helloworld');
console.log('✓ HelloWorld block created');

helloBlock.outerHTML

In [ ]:
// Example 2: Test Accordion block with content
const isNode = typeof process !== 'undefined' && process.versions?.node;
const testBlockFn = isNode ? global.testBlock : window.testBlock;

// Structure: Each row needs 2 divs (label and body)
const accordionContent = `
  <div>
    <div>What is EDS?</div>
    <div>Edge Delivery Services is Adobe's modern web platform for building fast, performant websites.</div>
  </div>
  <div>
    <div>How do blocks work?</div>
    <div>Blocks are JavaScript functions that decorate DOM elements and transform content structure.</div>
  </div>
  <div>
    <div>Why use JSLab?</div>
    <div>JSLab lets you experiment with blocks in a notebook environment for rapid testing and development.</div>
  </div>
  <div>
    <div>Why use ipynb-viewer?</div>
    <div>The ipynb-viewer block lets end users interact with executable notebooks directly on your EDS site.</div>
  </div>
`;

const accordionBlock = await testBlockFn('accordion', accordionContent);
console.log('✓ Accordion block created');

// Show stats
const doc = isNode ? global.document : document;
const detailsCount = accordionBlock.querySelectorAll('details').length;
console.log('Accordion sections:', detailsCount);

accordionBlock.outerHTML

## Part 3: Visual Output with Iframe

Create visual previews with iframe controls - works in both Node.js and browser!

In [ ]:
// Context-aware iframe preview generation
const isNode = typeof process !== 'undefined' && process.versions?.node;

const accordionContent = `
  <div>
    <div>What is EDS?</div>
    <div>Edge Delivery Services is Adobe's modern web platform for building fast, performant websites.</div>
  </div>
  <div>
    <div>How do blocks work?</div>
    <div>Blocks are JavaScript functions that decorate DOM elements and transform content structure.</div>
  </div>
  <div>
    <div>Why use JSLab?</div>
    <div>JSLab lets you experiment with blocks in a notebook environment for rapid testing and development.</div>
  </div>
  <div>
    <div>Why use ipynb-viewer?</div>
    <div>The ipynb-viewer block lets end users interact with executable notebooks directly on your EDS site.</div>
  </div>
`;

if (isNode) {
  // Node.js: Save files to disk
  await global.saveBlockHTML('accordion', accordionContent);
  
  console.log('\n✅ FILES CREATED (Node.js):');
  console.log('📂 ipynb-tests/accordion-preview.html - Styled block');
  console.log('📂 ipynb-tests/accordion-live-preview.html - Iframe with controls');
  console.log('🎨 Open accordion-live-preview.html in your browser!');
  
  'Files saved! Open accordion-live-preview.html in your browser';
} else {
  // Browser: Open in new window
  const testBlockFn = window.testBlock;
  const accordionBlock = await testBlockFn('accordion', accordionContent);
  
  // Open iframe preview
  window.openIframePreview('accordion', accordionBlock.outerHTML);
  
  console.log('\n✅ PREVIEW OPENED (Browser):');
  console.log('🎨 Iframe preview opened in new window');
  console.log('🖼️  Features: Refresh button, Close button (ESC key)');
  
  'Iframe preview opened in new window!';
}

## Part 4: Discover Available Blocks

List all blocks available for testing in this project.

In [ ]:
// Context-aware block discovery
const isNode = typeof process !== 'undefined' && process.versions?.node;

if (isNode) {
  // Node.js: Use fs to list directories
  const fs = require('fs').promises;
  const blocksDir = './blocks';
  
  const entries = await fs.readdir(blocksDir, { withFileTypes: true });
  const blocks = entries
    .filter(entry => entry.isDirectory())
    .map(entry => entry.name)
    .sort();
  
  console.log(`Found ${blocks.length} blocks:\n`);
  console.log(blocks.join('\n'));
  
  blocks;
} else {
  // Browser: Use fetch to get block list (if available)
  console.log('⚠ Block discovery requires Node.js/JSLab environment');
  console.log('ℹ️  In browser, blocks must be specified manually');
  
  // Return common blocks
  const commonBlocks = ['accordion', 'cards', 'columns', 'fragment', 'header', 'footer', 'hero'];
  console.log('Common blocks:', commonBlocks.join(', '));
  
  commonBlocks;
}

## Part 5: Interactive Calculations

Pure JavaScript that works in both environments.

In [ ]:
// Simple calculation - works everywhere
const a = 10;
const b = 20;
const sum = a + b;

console.log('Sum:', sum);
console.log('Product:', a * b);
console.log('Average:', sum / 2);

sum

In [ ]:
// Array operations - works everywhere
const numbers = [1, 2, 3, 4, 5];
const doubled = numbers.map(n => n * 2);
const sum = numbers.reduce((acc, n) => acc + n, 0);

console.log('Original:', numbers);
console.log('Doubled:', doubled);
console.log('Sum:', sum);

doubled

## Part 6: String Manipulation

In [ ]:
// Text transformations - works everywhere
const text = 'hello world from jupyter';
const upper = text.toUpperCase();
const titleCase = text.replace(/\b\w/g, c => c.toUpperCase());

console.log('Original:', text);
console.log('Uppercase:', upper);
console.log('Title Case:', titleCase);

titleCase

## Quick Reference

### Context Detection
```javascript
const isNode = typeof process !== 'undefined' && process.versions?.node;
const isBrowser = typeof window !== 'undefined';
```

### Context-Aware DOM Access
```javascript
const doc = isNode ? global.document : document;
const testBlockFn = isNode ? global.testBlock : window.testBlock;
```

### Node.js Helpers (JSLab only)

**Test blocks (returns DOM element):**
```javascript
const block = await global.testBlock('blockname', '<div>content</div>');
block.outerHTML
```

**Save with Live Preview (creates iframe wrapper):**
```javascript
// Creates TWO files:
// 1. blockname-preview.html (actual styled block)
// 2. blockname-live-preview.html (iframe wrapper with controls)
await global.saveBlockHTML('blockname', '<div>content</div>');

// Custom filename
await global.saveBlockHTML('blockname', '<div>content</div>', 'my-test.html');

// Disable live preview
await global.saveBlockHTML('blockname', '<div>content</div>', null, { livePreview: false });
```

**Create iframe preview HTML:**
```javascript
const previewHTML = global.createIframePreview('blockname', '<div>block html</div>');
```

**Load styles manually:**
```javascript
await global.loadBlockStyles('blockname');
```

### Browser Helpers (ipynb-viewer)

**Test blocks:**
```javascript
const block = await window.testBlock('blockname', '<div>content</div>');
const container = window.displayBlock(block);
```

**Create and open iframe preview:**
```javascript
// Generate iframe HTML
const previewHTML = window.createIframePreview('blockname', '<div>block html</div>');

// Open preview in new window
window.openIframePreview('blockname', '<div>block html</div>');
```

### Live Preview Features
- 🔴 Dark themed wrapper with controls
- ↻ Refresh button to reload preview
- ✕ Close button (or press ESC key)
- Status bar showing file location
- Fullscreen display with scrolling

### Tips
1. **Cell 1** auto-detects environment - always run it first
2. **JSLab mode**: Full block testing with jsdom, saves HTML files to disk
3. **Browser mode**: Direct DOM interaction, opens preview in new window
4. **Live preview**: Works in both environments with different mechanisms
5. **Context-aware**: Write code once, runs in both environments
6. **Node.js**: Files saved to `ipynb-tests/` directory
7. **Browser**: Preview opens in popup window via Blob URL

### Environment Capabilities

| Feature | JSLab (Node.js) | Browser (ipynb-viewer) |
|---------|----------------|------------------------|
| DOM Creation | ✅ jsdom | ✅ Native |
| Block Testing | ✅ Full | ⚠️ Limited |
| File I/O | ✅ Yes | ❌ No |
| Live Preview | ✅ Saved files | ✅ Popup window |
| Iframe Preview | ✅ Yes | ✅ Yes |
| Calculations | ✅ Yes | ✅ Yes |
| String Ops | ✅ Yes | ✅ Yes |